## Parking Controller via MPC with Obstacles

###System dynamics:
$z = [x,y,\phi]$, $u = [d_s, R, \beta]$
$d_s$ is the traveled staring distance,
$R$ is teh runing radius
$\beta$ is the traveled angle with turning rradius $R$


In [35]:
#Pkg.add("JuMP"); Pkg.add("Ipopt"); Pkg.add("Gadfly"); Pkg.add("Interact")
#Pkg.checkout("JuMP") # need latest master for matrices
using JuMP, Ipopt
using PyPlot
#using ODE
using PyCall

function solveMPC(l,w,n,m,T,z0,zT,zmin,zmax,umin,umax,dt,obs,eps)
     mpc = Model(solver=IpoptSolver(print_level=3))
    @defVar(mpc,  zmin[i] <= z[i=1:n,t=0:T] <= zmax[i])
    @defVar(mpc, -umax[i] <= u[i=1:m,t=0:T-1] <= umax[i])
    # Cost
    @setNLObjective(mpc, Min,
    sum{u[1,t]^2+(u[3,t]/(u[2,t]+eps))^2,t=0:T-1})
    
    # Link state and control across the horizon
    for t = 0:T-1        
        @addNLConstraint(mpc, z[1,t+1] == z[1,t] + u[1,t]*cos(z[3,t])-u[2,t]*sin(z[3,t])+u[2,t]*sin(z[3,t]+u[3,t]))
        @addNLConstraint(mpc, z[2,t+1] == z[2,t] + u[1,t]*sin(z[3,t])+u[2,t]*cos(z[3,t])-u[2,t]*cos(z[3,t]+u[3,t]))
        @addNLConstraint(mpc, z[3,t+1] == z[3,t] + u[3,t])
    end

    for t = 0:T-1 
        @addConstraint(mpc, u[2,t] >= umin[2])
    end
    # Radius constraint (it is an OR)
    if 1<0
        @defVar(mpc, lambda[i=1:2,t=0:T-1] >= 0)
        for t = 0:T-1 
            @addNLConstraint(mpc, lambda[1,t]*(u[2,t]-umin[2])+lambda[2,t]*(-u[2,t]-umin[2]) >=0)
            @addConstraint(mpc, lambda[1,t]+lambda[2,t] == 1)
        end
    end

    # Obstacle avoidance constraints -- NOT COMPLETED

    # Initial conditions
    @addConstraint(mpc, z[:,0] .== z0)
    # Final state
    @addConstraint(mpc, z[:,T] .== zT)
    # Solve the NLP
    solve(mpc)
    # Return the control plan
    # return getValue(u[:,0])
    #return getValue(u[:,0]), getValue(z[:,1])
    return getValue(u), getValue(z)
end

solveMPC (generic function with 1 method)

simulate_car_MPC (generic function with 1 method)

### Run Simulations

In [37]:
#Car Lengths to be fixed later
n=3
m=3
L=5-1-1.2
delta_max=25*pi/180
width=0.93
eps=0.001

z0 = [-1;5;0]  
zT = [0;2.5;pi]  
obs=[-3.5;3.5;3]
zmax=[20;20;8*pi]
zmin=[-20;-20;-8*pi]
T = 5 # MPC horizon
umax=[10;100;8*pi]
umin=[-umax[1];L/tan(delta_max)-eps;-umax[3]]
dt=100

print(umin[2])
# MPC
u_vec, z_vec= solveMPC(L,width,n,m,T,z0,zT,zmin,zmax,umin,umax,dt,obs,eps)



6.003619377426763

(u: 2 dimensions:
[1,:]
  [1,0] = 4.980366093937434e-5
  [1,1] = -8.033752155876962e-5
  [1,2] = -3.3827197644755054e-5
  [1,3] = -5.6693852557877723e-5
  [1,4] = -6.140311827759966e-5
[2,:]
  [2,0] = 22.199847950575478
  [2,1] = 99.99994783361386
  [2,2] = 24.831547945169973
  [2,3] = 99.99993119513572
  [2,4] = 22.042509736187508
[3,:]
  [3,0] = 1.2405035203858445
  [3,1] = -0.40438601872884844
  [3,2] = 1.5430442860453744
  [3,3] = -0.3744769724955687
  [3,4] = 1.1369078383829914
,z: 2 dimensions:
[1,:]
  [1,0] = -1.0
  [1,1] = 19.999938721437097
  [1,2] = -0.3902097194806473
  [1,3] = -1.6656975009022073
  [1,4] = 19.999978340248028
  [1,5] = 0.0
[2,:]
  [2,0] = 5.0
  [2,1] = 19.999992780310308
  [2,2] = -14.602949354108352
  [2,3] = 19.999987719910113
  [2,4] = -10.275735655270427
  [2,5] = 2.5
[3,:]
  [3,0] = -2.8186096220332553e-34
  [3,1] = 1.2405035203858445
  [3,2] = 0.8361175016569962
  [3,3] = 2.3791617877023707
  [3,4] = 2.0046848152068018
  [3,5] = 3.141592653589793
)

###Next cell plots the parking manouver

In [1]:
@pyimport matplotlib.pyplot as plt
print(z_vec[:,[end]])

#Find Axis Limits
xmin = minimum([z_vec[1,:]])
xmax = maximum([z_vec[1,:]])
xmin, xmax = xmin - 0.1(xmax-xmin), xmax+ 0.1*(xmax-xmin)
ymin = minimum([z_vec[2,:]])
ymax = maximum([z_vec[2,:]])
ymin, ymax = ymin - 0.1(ymax-ymin), ymax+ 0.1*(ymax-ymin)

function plot_car(x,y,psi,beta,l,w,pt)
    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-l*cos(psi)+w*sin(psi),y-l*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-l*cos(psi)-w*sin(psi),y-l*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.4
    E=[A[1]+lw*cos(psi+beta),A[2]+lw*sin(psi+beta)]
    F=[C[1]+lw*cos(psi+beta),C[2]+lw*sin(psi+beta)]
    
    pt.plot([B[1] ,A[1]],[B[2], A[2]],"bo-")
    pt.plot([C[1] ,D[1]],[C[2], D[2]],"bo-")
    pt.plot([C[1] ,A[1]],[C[2], A[2]],"bo-")
    pt.plot([B[1] ,D[1]],[B[2], D[2]],"bo-")
    pt.plot([A[1] ,E[1]],[A[2], E[2]],"go-")
    pt.plot([C[1] ,F[1]],[C[2], F[2]],"go-")
end


#Construct Figure and Plot Data
fig = figure()
ax = plt.axes()
ax = plt.axes(xlim = (-20,20),ylim=(-2,20))
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
plt.plot(transpose(z_vec[1,:]),transpose(z_vec[2,:]), "r-")

#Plot sow in TIME
for i=1:1:T
    plot_car(z_vec[1,i],z_vec[2,i],z_vec[3,i],u_vec[1,i],l,width,plt)
end
plot_car(z_vec[1,[end]],z_vec[2,[end]],z_vec[3,[end]],u_vec[1,[end]],l,width,plt)



##end#Plot in SPACE
#NtP=6
#u_history = zeros(m,NtP*Tsim)
#z_history = zeros(n,NtP*Tsim)
#k=1
#for i=1:T
#    z_t=z_vec[:,i]
#    u_t=u_vec[:,i]
#    ddt=0.#5
 #   for j=0:ddt:dt
#        plot_car(z_t[1],z_t[2],z_t[3],u_t[1],l,width,plt)
#        z_history[:,k] = z_t[:]
 #       u_history[:,k] = u_t[:]
 #       z_t = z_t + ddt*zdot_fun(z_t,u_t,l)
 #       k=k+1
 #   en#d
#end
plt.plot([obs[1] ,obs[1]],[0, obs[3]],"r-")
plt.plot([obs[2] ,obs[2]],[0, obs[3]],"r-")
plt.plot([-20 ,obs[1]],[obs[3], obs[3]],"r-")
plt.plot([obs[2] ,20],[obs[3], obs[3]],"r-")
plt.plot([-20 ,20],[zmax[2]+1, zmax[2]+1],"r-")
plt.show()


#Plot start and end points
#ax[:plot](z_vec[1,1],z_vec[2,1], "ro")
#ax[:plot](z_vec[1,end],z_vec[2,end], "rs")

LoadError: @pyimport not defined
while loading In[1], in expression starting on line 2

### Next makes a video of the manouver

In [9]:
#Pkg.add("PyPlot")
#Pkg.add("PyCall")
#Pkg.add("VideoIO")
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim


pygui(true)

# First set up the figure, the axis, and the plot element we want to animate
fig = figure()
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
#ax = plt.axes(xlim=(-10, 5), ylim=(-2, 10))
ax = plt.axes(xlim=(-5, 5), ylim=(-0, 10))
#ax2 = plt.axes()
global line1 = ax[:plot]([], [], "ro-")[1]
global line2 = ax[:plot]([], [], "ro-")[1]
global line3 = ax[:plot]([], [], "ro-")[1]
global line4 = ax[:plot]([], [], "ro-")[1]
global line5 = ax[:plot]([], [], "go-")[1]
global line6 = ax[:plot]([], [], "go-")[1]
ax[:plot]([obs[1] ,obs[1]],[0, obs[3]],"r-")
ax[:plot]([obs[2] ,obs[2]],[0, obs[3]],"r-")
ax[:plot]([-20 ,obs[1]],[obs[3], obs[3]],"r-")
ax[:plot]([obs[2] ,20],[obs[3], obs[3]],"r-")
ax[:plot]([-20 ,20],[zmax[2]+1, zmax[2]+1],"r-")
#plt.plot([-3 ,-3],[-3, 0],"r-")
#plt.plot([3 ,3],[-3, 0],"r-")
#plt.plot([-10 ,-3],[0, 0],"r-")
#plt.plot([3 ,15],[0, 0],"r-")

function points_car(x,y,psi,beta,l,w)
    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-l*cos(psi)+w*sin(psi),y-l*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-l*cos(psi)-w*sin(psi),y-l*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.6
    E=[A[1]+lw/2*cos(psi+beta),A[2]+lw/2*sin(psi+beta)]
    F=[C[1]+lw/2*cos(psi+beta),C[2]+lw/2*sin(psi+beta)]
    G=[A[1]-lw/2*cos(psi+beta),A[2]-lw/2*sin(psi+beta)]
    H=[C[1]-lw/2*cos(psi+beta),C[2]-lw/2*sin(psi+beta)]
    return A,B,C,D,E,F,G,H
end
 

# initialization function: plot the background of each frame
function init()
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    global line7
    line1[:set_data]([], [])
    line2[:set_data]([], [])
    line3[:set_data]([], [])
    line4[:set_data]([], [])
    line5[:set_data]([], [])
    line6[:set_data]([], [])
    return (line1,line2,line3,line4,line5,line6,None)
end

# animation function.  This is called sequentially
function animate(i)
    k=i+1
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    global line7
    if k<=T
        A,B,C,D,E,F,G,H=points_car(z_vec[1,k],z_vec[2,k],z_vec[3,k],u_vec[1,k],l,width)
    else
        A,B,C,D,E,F,G,H=points_car(z_vec[1,T],z_vec[2,T],z_vec[3,T],u_vec[1,T],l,width)
    end
    line1[:set_data]([B[1] ,A[1]],[B[2], A[2]])
    line2[:set_data]([C[1] ,D[1]],[C[2], D[2]])
    line3[:set_data]([C[1] ,A[1]],[C[2], A[2]])
    line4[:set_data]([B[1] ,D[1]],[B[2], D[2]])
    line5[:set_data]([G[1] ,E[1]],[G[2], E[2]])
    line6[:set_data]([H[1] ,F[1]],[H[2], F[2]])
    return (line1,line2,line3,line4,line5,line6,None)
end


# call the animator.  blit=True means only re-draw the parts that have changed.
myanim = anim.FuncAnimation(fig, animate, init_func=init, frames=T+20,  interval=100,)
myanim[:save]("parking.mp4", writer="ffmpeg",  extra_args=["-vcodec", "libx264"])
myanim[:save]("parking2.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])


LoadError: key not found: "Rectangle"
while loading In[9], in expression starting on line 23